In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN4UZVE/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN4UZVE/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
dataset

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [5]:
train_data = dataset['train']
test_data = dataset['test']

In [9]:
for sentence, label in train_data:
  print(sentence)
  print(label)
  break

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [10]:
train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for sentence, label in train_data:
  train_sentences.append(sentence.numpy().decode('utf8'))
  train_labels.append(label.numpy())

for sentence, label in test_data:
  test_sentences.append(sentence.numpy().decode('utf8'))
  test_labels.append(label.numpy())

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [16]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_token = '<oov>'

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

In [49]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                                    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(8, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional_6 (Bidirectio  (None, 32)               4224      
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 8)                 264       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 164,497
Trainable params: 164,497
Non-trainable params: 0
________________________________________________

In [50]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)

In [54]:
history = model.fit(padded_sequences,
                    train_labels,
                    validation_data=(padded_test_sequences, test_labels),
                    epochs=10,
                    callbacks=early_stopping)

Epoch 1/10
782/782 [==============================] - 102s 130ms/step - loss: 0.3882 - accuracy: 0.8305 - val_loss: 0.3977 - val_accuracy: 0.8264
Epoch 2/10
782/782 [==============================] - 45s 58ms/step - loss: 0.2743 - accuracy: 0.8924 - val_loss: 0.4114 - val_accuracy: 0.8209
Epoch 3/10
782/782 [==============================] - 45s 58ms/step - loss: 0.2205 - accuracy: 0.9169 - val_loss: 0.5115 - val_accuracy: 0.8067
Epoch 4/10
782/782 [==============================] - 45s 58ms/step - loss: 0.1747 - accuracy: 0.9346 - val_loss: 0.5879 - val_accuracy: 0.8118


In [55]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

(10000, 16)


In [56]:
# http://projector.tensorflow.org/
# Upload meta.tsv, vecs.tsv

In [57]:
model.save('imdb_reviews.h5')